In [1]:
from datetime import datetime, timedelta

In [2]:
def find_closest_saturday_improved(start_date):
    day_of_week = start_date.weekday()
    if day_of_week == 5:
        return start_date
    distance_to_last_saturday = (start_date.weekday() - 5) % 7
    distance_to_next_saturday = (5 - start_date.weekday()) % 7
    if distance_to_last_saturday <= distance_to_next_saturday:
        return start_date - timedelta(days=distance_to_last_saturday)
    return start_date + timedelta(days=distance_to_next_saturday)


def find_closest_sunday(date):
    day_of_week = date.weekday()
    distance_to_next_sunday = (6 - day_of_week) % 7
    return date + timedelta(days=distance_to_next_sunday)


def add_one_month_plus_one_day(date):
    month = (date.month % 12) + 1
    year = date.year + (date.month // 12)
    try:
        return datetime(year, month, date.day + 1)
    except ValueError:
        return datetime(year, month, 28) + timedelta(days=1)


def find_closest_monday(date):
    day_of_week = date.weekday()
    distance_to_next_monday = (7 - day_of_week) % 7
    return date + timedelta(days=distance_to_next_monday)


def calculate_module_dates_custom_start_corrected(start_date, num_modules):
    module_dates = []
    evaluation_dates = []
    rectification_dates = []
    platform_close_dates = []

    if num_modules == 1:
        for i in range(4):
            module_date = find_closest_saturday_improved(
                start_date + timedelta(days=7 * i)
            )
            module_dates.append(module_date)
            evaluation_start = module_dates[-1] + timedelta(days=7)
            evaluation_end = evaluation_start + timedelta(days=4)
            evaluation_dates.append((evaluation_start, evaluation_end))
            rectification_start = evaluation_end + timedelta(days=1)
            rectification_end = rectification_start + timedelta(days=3)
            rectification_dates.append((rectification_start, rectification_end))
            platform_close_date = find_closest_monday(rectification_end)
            platform_close_dates.append(platform_close_date)
    else:
        for i in range(num_modules):
            if i == 0:
                module_date = find_closest_saturday_improved(start_date)
            else:
                month = (module_date.month % 12) + 1
                year = module_date.year + (module_date.month // 12)

                try:
                    next_month_date = datetime(year, month, start_date.day)
                except ValueError:
                    next_month_date = datetime(year, month, 28)

                module_date = find_closest_saturday_improved(next_month_date)

            module_dates.append(module_date)
            evaluation_start = add_one_month_plus_one_day(module_date)
            evaluation_end = evaluation_start + timedelta(days=14)
            evaluation_dates.append((evaluation_start, evaluation_end))

            rectification_start = evaluation_end + timedelta(days=1)
            rectification_end = rectification_start + timedelta(days=14)
            rectification_dates.append((rectification_start, rectification_end))

            platform_close_date = find_closest_sunday(rectification_end)
            platform_close_dates.append(platform_close_date)

    return module_dates, evaluation_dates, rectification_dates, platform_close_dates


def determine_week_of_month(date):
    first_day_of_month = date.replace(day=1)
    day_of_month = date.day
    adjusted_dom = day_of_month + first_day_of_month.weekday()
    return int(adjusted_dom / 7) + (1 if adjusted_dom % 7 > 0 else 0)

In [3]:
# Definiciones de días y meses en español
days_es = {
    "Monday": "Lunes", "Tuesday": "Martes", "Wednesday": "Miércoles",
    "Thursday": "Jueves", "Friday": "Viernes", "Saturday": "Sábado", "Sunday": "Domingo"
}
months_es = {
    "January": "Enero", "February": "Febrero", "March": "Marzo", "April": "Abril",
    "May": "Mayo", "June": "Junio", "July": "Julio", "August": "Agosto",
    "September": "Septiembre", "October": "Octubre", "November": "Noviembre", "December": "Diciembre"
}

# Función para formatear fechas en español
def format_date_spanish(date):
    day_spanish = days_es[date.strftime("%A")]
    month_spanish = months_es[date.strftime("%B")]
    return f"{day_spanish}, {date.strftime('%d')} de {month_spanish} del {date.strftime('%Y')}"

# Asumiendo que calculate_module_dates_custom_start_corrected y determine_week_of_month están definidas

# Input
start_date = datetime(2024, 3, 9)
num_modules = 5

module_dates, evaluation_dates, rectification_dates, platform_close_dates = calculate_module_dates_custom_start_corrected(start_date, num_modules)

# Impresión de información general del programa
print("Fecha de inicio del programa:", format_date_spanish(start_date))
program_end_date = evaluation_dates[-1][0] - timedelta(days=1)
print(f"Fecha de término del programa: {format_date_spanish(program_end_date)}\n")

# Detalles sobre los módulos
week_of_month = determine_week_of_month(module_dates[0])
week_label = ["1era", "2da", "3ra", "4ta", "5ta"]
print(f"El módulo comienza en la {week_label[week_of_month-1]} semana del mes.")
month_year_module_1 = f"{months_es[module_dates[0].strftime('%B')]} del {module_dates[0].strftime('%Y')}"
print(f"Mes y año del módulo: {month_year_module_1}\n")

# Impresión de fechas para cada módulo y sus periodos relevantes
for i, (module_date, evaluation_period, rectification_period, close_date) in enumerate(zip(module_dates, evaluation_dates, rectification_dates, platform_close_dates), start=1):
    print(f"Módulo {i}: {format_date_spanish(module_date)}")
print(f"Período de evaluación final: Inicia el {format_date_spanish(evaluation_period[0])}, finaliza el {format_date_spanish(evaluation_period[1])}")
print(f"Período de subsanación: Inicia el {format_date_spanish(rectification_period[0])}, finaliza el {format_date_spanish(rectification_period[1])}")
print(f"Cierre de acceso a la plataforma: {format_date_spanish(close_date)}\n")


Fecha de inicio del programa: Sábado, 09 de Marzo del 2024
Fecha de término del programa: Martes, 06 de Agosto del 2024

El módulo comienza en la 2da semana del mes.
Mes y año del módulo: Marzo del 2024

Módulo 1: Sábado, 09 de Marzo del 2024
Módulo 2: Sábado, 06 de Abril del 2024
Módulo 3: Sábado, 11 de Mayo del 2024
Módulo 4: Sábado, 08 de Junio del 2024
Módulo 5: Sábado, 06 de Julio del 2024
Período de evaluación final: Inicia el Miércoles, 07 de Agosto del 2024, finaliza el Miércoles, 21 de Agosto del 2024
Período de subsanación: Inicia el Jueves, 22 de Agosto del 2024, finaliza el Jueves, 05 de Septiembre del 2024
Cierre de acceso a la plataforma: Domingo, 08 de Septiembre del 2024

